In [1]:
# Standard library imports
import os

# Third-party library imports
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the GameSim API key
# .env file format:
# GAMESIM_API_KEY="<api key>"
api_key = os.getenv('GAMESIM_API_KEY')

base_gamesim_url = 'https://api.ncaagamesim.com'

# Usage is explained in "Potential Bugs/Feature Requests" Number 1.
postman_user_agent = 'PostmanRuntime/7.33.0'

# Questions

### 1. ncaaScheduleId/nflScheduleId

The request data for a simulation post is as follows:

```
request_data = {
    "hometeam": "homeTeam",
    "homeyear": "homeYear",
    "awayteam": "awayTeam",
    "awayyear": "awayYear",
    "includeStats": True,
    "ncaaScheduleId": 0,
    "nflScheduleId": 0,
    "numberOfSimulations": 0
}
```

Curious what 'ncaaScheduleId'/'nflScheduleId' represent or control? With some basic testing I was able to pass various integers into those fields and couldn't see any difference in the returned data.

# Feature Requests

### 1. API Usage Endpoint/Field

Currently there doesn't appear to be a way to see how many calls you have made to the API in the given billing timeframe. This would be very helpful, weather it's apart of each response (prefered in my opinion) or a seperate endput.

### 2. Home Field Advantage and Show Play-by-Play fields

Would love to have access to these options from the API.
```
request_data = {
    "hometeam": "Illinois",
    "homeyear": "2024",
    "awayteam": "Iowa",
    "awayyear": "2024",
    "includeStats": True,
    "ncaaScheduleId": 0,
    "nflScheduleId": 0,
    "numberOfSimulations": 0,
    "homeFieldAdvantage": 1,
    "showPlayByPlay": 1
}

sport = 'cfb'

simulate_response = requests.post(base_gamesim_url + f'/gamesim/simulate/{sport}', headers=headers, json=request_data)
```

### 3. Predictions without time or by week

Would love to be able to get all predictions for a given league without passing a time, e.g all NFL predictions on 10/17/2023 OR even better all predictions for a given league (probably just football) by week, e.g all CFB predictions for Week 7. Full week could be done fairly easily if we could call predictions with only a date by paging through that week, but with times it makes it difficult unless that data also became avalible through the API.


Date Only:
```
sport = 'cfb'
date = '2023-10-17'

prediction_response = requests.get(base_gamesim_url + f'/gamesim/predictions/{sport}/{date}', headers=headers)
```


Week Only:
```
# Example for Week Only
sport = 'cfb'
week = '7'

prediction_response = requests.get(base_gamesim_url + f'/gamesim/predictions/{sport}/{week}', headers=headers)
```

# Potential Bugs

### 1. Only certain User-Agents have Access Permisions

### Without Set User-Agent:

In [2]:
headers = {
    'Authorization': api_key
}

response = requests.get('https://api.ncaagamesim.com/gamesim/ping', headers=headers)
print(response.status_code, response.reason)

403 ModSecurity Action


We can use [hhtpbin](https://httpbin.org/) to easily see the auto-generated headers of our request.

In [3]:
request_with_all_headers = requests.get('http://httpbin.org/headers') # Note no headers are included in this request.
print(f"Auto-Generated User-Agent: {request_with_all_headers.json()['headers']['User-Agent']}")

Auto-Generated User-Agent: python-requests/2.28.1


Python automatically assigned the User-Agent to "python-requests/2.28.1", which was denied access.

Since we know this request has worked in Postman before, let's try setting 'User-Agent' to the Postman value.

### With Set User-Agent:

In [4]:
headers = {
    'Authorization': api_key,
    'User-Agent': 'PostmanRuntime/7.33.0'
}

response = requests.get('https://api.ncaagamesim.com/gamesim/ping', headers=headers)
print(response.status_code, response.reason)

200 OK


Assume this is intended on some level, would be curious if possible to expand the User-Agents with permissions, especially because it is easy to spoof the User-Agent. Not a huge deal at all. 


Tested with the following User-Agents as well:

|          User-Agent      |      Response Status       |
| :-----------------------:| :------------------------: |
|  PostmanRuntime/7.33.0   |    200 OK                  |
|  Boomi Http Transport*   |    200 OK                  |  
|  python-requests/2.28.1  |    403 ModSecurity Action  |
|  python-requests/2.30.0  |    403 ModSecurity Action  |

*User-Agent auto-generated by Boomi